# Introductin

This dataset has a constant wirefeed and travelspeed. The only difference is the welding current. Only every second layer is measured !

In [9]:

import weldTransform as wt
import os
import pandas as pd

# Loading the dataset

In [10]:
readColumns = ['timestamp','RIst/X','RIst/Y','RIst/Z','MD/LSZ','PD/WF',"WD/W04",'PD/FR','PD/MO','PD/LA','PD/I','WD/W02','WD/W03']
outputColumns = ['timestamp','X','Y','Z','Z-Height','WirefeedSet',"WirefeedReal",'Feedrate','Mode','Layer',"CurrentSet",'CurrentReal','VoltageReal']
filePath = r'..\data\Dataset 2\Versuch P10118.4\20241121_1229_Daten_P10118.csv' 
print(os.getcwd())
df = wt.load_dataset(filePath, readColumns, outputColumns)

#flip x and y colums so its aligned with the first dataset 
df = df.rename(columns={'X': 'Y', 'Y': 'X'})


c:\Users\stefa\Desktop\DSProject\PMD-Process-Analysis\src


In [11]:
# check unique layers for mode = 2
print(df[df['Mode'] == 2]['Layer'].unique())
#check unique layers for mode = 4
print(df[df['Mode'] == 4]['Layer'].unique())

[1. 2. 3. 4. 5. 6. 7. 8.]
[0. 1. 2. 3. 4. 5. 6. 7. 8.]


# Preprocessing and Data Exploratiton

From Y axis along the weld we remove the start and end parts. And for the x axis across the seam we remove the points that where tracked on previous layers.

In [12]:
# Visualiziing before preprocessing

wt.plot2D_crossection(df,4,"Y")
wt.plot2D_crossection(df,5)

In [13]:
#processing x and y
df_processed_yx = wt.filter_mode_5_crossection_points(wt.cutoff_start_end_y(df),0.2)


In [28]:
wt.plot2D_crossection(df_processed_yx,4,"Y")
wt.plot2D_crossection(df_processed_yx,5)

From the bottom plot we can see that some data is missing since some overhangs were created. And on the right some splattering.

Lets check mode 2.

In [15]:
wt.plot2D_crossection(df_processed_yx,2,"Y","Z")

Now lets check the current set for each layer

In [26]:
grouped = df_processed_yx[df_processed_yx["Mode"] == 2].groupby("Layer")["CurrentSet"].unique()
print(grouped)


Layer
1.0    [270.0]
2.0    [240.0]
3.0    [230.0]
4.0    [225.0]
5.0    [225.0]
6.0    [225.0]
7.0    [225.0]
8.0    [225.0]
Name: CurrentSet, dtype: object


Now lets plot the observed voltage of time on each weld.

In [27]:
wt.plot2D_crossection(df_processed_yx,2,"Y","VoltageReal")

From this plot you can see that since each layer has two passes two diverging signals can be spotted.

# Syncing up the weld data with the measurement data

Same as previous

In [ ]:
df_processed_yx_synced = wt.sync_measurement_with_weld(df_processed_yx,passesPerMeasurement=2)